In [108]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Cities.csv
Conferences.csv
MConferenceTourneyGames.csv
MGameCities.csv
MMasseyOrdinals.csv
MNCAATourneyCompactResults.csv
MNCAATourneyDetailedResults.csv
MNCAATourneySeedRoundSlots.csv
MNCAATourneySeeds.csv
MNCAATourneySlots.csv
MRegularSeasonCompactResults.csv
MRegularSeasonDetailedResults.csv
MSampleSubmissionStage1.csv
MSeasons.csv
MSecondaryTourneyCompactResults.csv
MSecondaryTourneyTeams.csv
MTeamCoaches.csv
MTeamConferences.csv
MTeamSpellings.csv
MTeams.csv



In [109]:
#Merges in seeds and and makes the sample submission

In [110]:
#the seed information
df_seeds = pd.read_csv('../input/MNCAATourneySeeds.csv')

#tour information
df_tour = pd.read_csv('../input/MRegularSeasonCompactResults.csv')

In [111]:
df_seeds['seed_int'] = df_seeds['Seed'].apply( lambda x : int(x[1:3]) )

df_winseeds = df_seeds.loc[:, ['TeamID', 'Season', 'seed_int']].rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.loc[:, ['TeamID', 'Season', 'seed_int']].rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])

In [112]:
df_concat['DiffSeed'] = df_concat[['LSeed', 'WSeed']].apply(lambda x : 0 if x[0] == x[1] else 1, axis = 1)

In [113]:
#prepares sample submission
df_sample_sub = pd.read_csv('../input/MSampleSubmissionStage1.csv')

In [114]:
df_sample_sub['Season'] = df_sample_sub['ID'].apply(lambda x : int(x.split('_')[0]) )
df_sample_sub['TeamID1'] = df_sample_sub['ID'].apply(lambda x : int(x.split('_')[1]) )
df_sample_sub['TeamID2'] = df_sample_sub['ID'].apply(lambda x : int(x.split('_')[2]) )

In [115]:
df_sample_sub

,ID,Pred,Season,TeamID1,TeamID2
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129
...,...,...,...,...,...
11385,2019_1449_1459,0.5,2019,1449,1459
11386,2019_1449_1463,0.5,2019,1449,1463
11387,2019_1458_1459,0.5,2019,1458,1459
11388,2019_1458_1463,0.5,2019,1458,1463


# Training Data Creation

In [116]:
#this makes dataframe with reverse losers and winners

In [117]:
winners = df_concat.rename( columns = { 'WTeamID' : 'TeamID1', 
                                                       'LTeamID' : 'TeamID2',
                                                      'WScore' : 'Team1_Score',
                                                      'LScore' : 'Team2_Score'}).drop(['WSeed', 'LSeed', 'WLoc'], axis = 1)
winners['Result'] = 1.0

losers = df_concat.rename( columns = { 'WTeamID' : 'TeamID2', 
                                                       'LTeamID' : 'TeamID1',
                                                      'WScore' : 'Team2_Score',
                                                      'LScore' : 'Team1_Score'}).drop(['WSeed', 'LSeed', 'WLoc'], axis = 1)

losers['Result'] = 0.0

train = pd.concat( [winners, losers], axis = 0).reset_index(drop = True)

train['Score_Ratio'] = train['Team1_Score'] / train['Team2_Score']
train['Score_Total'] = train['Team1_Score'] + train['Team2_Score']
train['Score_Pct'] = train['Team1_Score'] / train['Score_Total']

In [118]:
years = [2015, 2016, 2017, 2018,2019]

In [119]:
#this merges so only the games that we actually need to predict our in the new frame

In [120]:
train_test_inner = pd.merge( train.loc[ train['Season'].isin(years), : ].reset_index(drop = True), 
         df_sample_sub.drop(['ID', 'Pred'], axis = 1), 
         on = ['Season', 'TeamID1', 'TeamID2'], how = 'inner' )

In [121]:
train_test_inner.head()

,Season,DayNum,TeamID1,Team1_Score,TeamID2,Team2_Score,NumOT,DiffSeed,Result,Score_Ratio,Score_Total,Score_Pct
0,2015,57,1173,78,1279,74,0,0,1.0,1.054054,152,0.513158
1,2015,64,1246,89,1279,86,1,1,1.0,1.034884,175,0.508571
2,2015,72,1261,75,1279,71,0,1,1.0,1.056338,146,0.513699
3,2015,117,1261,73,1279,63,0,1,1.0,1.158730,136,0.536765
4,2015,78,1208,69,1279,64,0,1,1.0,1.078125,133,0.518797


In [122]:
team1d_num_ot = train_test_inner.groupby(['Season', 'TeamID1'])['NumOT'].median().reset_index()\
.set_index('Season').rename(columns = {'NumOT' : 'NumOT1'})
team2d_num_ot = train_test_inner.groupby(['Season', 'TeamID2'])['NumOT'].median().reset_index()\
.set_index('Season').rename(columns = {'NumOT' : 'NumOT2'})

num_ot = team1d_num_ot.join(team2d_num_ot).reset_index()

#sum the number of ot calls and subtract by one to prevent overcounting
num_ot['NumOT'] = num_ot[['NumOT1', 'NumOT2']].apply(lambda x : round( x.sum() ), axis = 1 )

num_ot

,Season,TeamID1,NumOT1,TeamID2,NumOT2,NumOT
0,2015,1107,0.0,1139,0.0,0.0
1,2015,1107,0.0,1173,0.0,0.0
2,2015,1107,0.0,1207,0.0,0.0
3,2015,1107,0.0,1208,0.0,0.0
4,2015,1107,0.0,1211,0.0,0.0
...,...,...,...,...,...,...
14179,2019,1439,0.0,1439,0.0,0.0
14180,2019,1439,0.0,1449,0.0,0.0
14181,2019,1439,0.0,1458,0.0,0.0
14182,2019,1439,0.0,1459,0.0,0.0


Here we look at the comparable statistics.  For the TeamID2 column, we would consider the inverse of the ratio, and 1 minus the score attempt percentage.

In [123]:
def geo_mean( x ):
    return np.exp( np.mean(np.log(x)) )

def harm_mean( x ):
    return np.mean( x ** -1.0 ) ** -1.0

team1d_score_spread = train_test_inner.groupby(['Season', 'TeamID1'])[['Score_Ratio', 'Score_Pct']]\
.agg({ 'Score_Ratio': geo_mean, 'Score_Pct' : harm_mean}).reset_index()\
.set_index('Season').rename(columns = {'Score_Ratio' : 'Score_Ratio1', 'Score_Pct' : 'Score_Pct1'})
team2d_score_spread = train_test_inner.groupby(['Season', 'TeamID2'])[['Score_Ratio', 'Score_Pct']]\
.agg({ 'Score_Ratio': geo_mean, 'Score_Pct' : harm_mean}).reset_index()\
.set_index('Season').rename(columns = {'Score_Ratio' : 'Score_Ratio2', 'Score_Pct' : 'Score_Pct2'})

score_spread = team1d_score_spread.join(team2d_score_spread).reset_index()

#geometric mean of score ratio of team 1 and inverse of team 2
score_spread['Score_Ratio'] = score_spread[['Score_Ratio1', 'Score_Ratio2']].apply(lambda x : ( x[0] * ( x[1] ** -1.0) ), axis = 1 ) ** 0.5

#harmonic mean of score pct
score_spread['Score_Pct'] = score_spread[['Score_Pct1', 'Score_Pct2']].apply(lambda x : 0.5*( x[0] ** -1.0 ) + 0.5*( 1.0 - x[1] ) ** -1.0, axis = 1 ) ** -1.0

score_spread.head()

,Season,TeamID1,Score_Ratio1,Score_Pct1,TeamID2,Score_Ratio2,Score_Pct2,Score_Ratio,Score_Pct
0,2015,1107,0.9375,0.483871,1139,0.835821,0.455285,1.059081,0.512494
1,2015,1107,0.9375,0.483871,1173,1.268858,0.559234,0.859566,0.461314
2,2015,1107,0.9375,0.483871,1207,0.986671,0.495765,0.974764,0.493843
3,2015,1107,0.9375,0.483871,1208,1.135692,0.531063,0.908564,0.476287
4,2015,1107,0.9375,0.483871,1211,0.935126,0.482059,1.001269,0.500327


Now lets create a model just solely based on the inner group and predict those probabilities. 

We will get the teams with the missing result.

In [124]:
X_train = train_test_inner.loc[:, ['Season', 'NumOT', 'Score_Ratio', 'Score_Pct']]
train_labels = train_test_inner['Result']

train_test_outer = pd.merge( train.loc[ train['Season'].isin(years), : ].reset_index(drop = True), 
         df_sample_sub.drop(['ID', 'Pred'], axis = 1), 
         on = ['Season', 'TeamID1', 'TeamID2'], how = 'outer' )

train_test_outer = train_test_outer.loc[ train_test_outer['Result'].isnull(), 
                                        ['TeamID1', 'TeamID2', 'Season']]

train_test_missing = pd.merge( pd.merge( score_spread.loc[:, ['TeamID1', 'TeamID2', 'Season', 'Score_Ratio', 'Score_Pct']], 
                   train_test_outer, on = ['TeamID1', 'TeamID2', 'Season']),
         num_ot.loc[:, ['TeamID1', 'TeamID2', 'Season', 'NumOT']],
         on = ['TeamID1', 'TeamID2', 'Season'])

In [125]:
train_test_missing

,TeamID1,TeamID2,Season,Score_Ratio,Score_Pct,NumOT
0,1107,1139,2015,1.059081,0.512494,0.0
1,1107,1173,2015,0.859566,0.461314,0.0
2,1107,1207,2015,0.974764,0.493843,0.0
3,1107,1208,2015,0.908564,0.476287,0.0
4,1107,1211,2015,1.001269,0.500327,0.0
...,...,...,...,...,...,...
8088,1438,1459,2019,0.985909,0.492140,0.0
8089,1438,1463,2019,1.025304,0.507038,0.0
8090,1439,1458,2019,1.104502,0.525934,0.0
8091,1439,1459,2019,0.982916,0.491967,0.0


We scale our data for our logistic regression, and make sure our categorical variables are properly processed.

In [126]:
X_test = train_test_missing.loc[:, ['Season', 'NumOT', 'Score_Ratio', 'Score_Pct']]

n = X_train.shape[0]

train_test_merge = pd.concat( [X_train, X_test], axis = 0 ).reset_index(drop = True)

train_test_merge = pd.concat( [pd.get_dummies( train_test_merge['Season'].astype(object) ), 
            train_test_merge.drop('Season', axis = 1) ], axis = 1 )

train_test_merge = pd.concat( [pd.get_dummies( train_test_merge['NumOT'].astype(object) ), 
            train_test_merge.drop('NumOT', axis = 1) ], axis = 1 )

X_train = train_test_merge.loc[:(n - 1), :].reset_index(drop = True)
X_test = train_test_merge.loc[n:, :].reset_index(drop = True)

In [127]:
x_max = X_train.max()
x_min = X_train.min()

X_train = ( X_train - x_min ) / ( x_max - x_min + 1e-14)
X_test = ( X_test - x_min ) / ( x_max - x_min + 1e-14)

In [128]:
X_train

,0.0,1.0,2.0,3.0,4.0,2015,2016,2017,2018,2019,Score_Ratio,Score_Pct
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.413923,0.613307
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.402568,0.602101
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.415276,0.614628
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.475927,0.670984
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.428181,0.627084
...,...,...,...,...,...,...,...,...,...,...,...,...
1557,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.278388,0.464195
1558,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.315162,0.508226
1559,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.328055,0.522987
1560,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.287130,0.474931


In [129]:
from sklearn.linear_model import LogisticRegressionCV

log_clf = LogisticRegressionCV(cv = 5)

log_clf.fit( X_train, train_labels )

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

LogisticRegressionCV(cv=5)

Here we store our probabilities

In [130]:
train_test_inner['Pred1'] = log_clf.predict_proba(X_train)[:,1]
train_test_missing['Pred1'] = log_clf.predict_proba(X_test)[:,1]

We merge our predictions

In [150]:
sub = pd.merge(df_sample_sub, 
                         pd.concat( [train_test_missing.loc[:, ['Season', 'TeamID1', 'TeamID2', 'Pred1']],
                                     train_test_inner.loc[:, ['Season', 'TeamID1', 'TeamID2', 'Pred1']] ],
                                   axis = 0).reset_index(drop = True),
                  on = ['Season', 'TeamID1', 'TeamID2'], how = 'outer')

In [151]:
sub = sub.groupby(["TeamID1","TeamID2","Season"]).agg({"Pred":"mean","Pred1":"mean","ID":"first"}).reset_index()

We get the 'average' probability of success for each team

In [152]:
team1_probs = sub.groupby('TeamID1')['Pred1'].apply(lambda x : (x ** -1.0).mean() ** -1.0 ).fillna(0.5).to_dict()
team2_probs = sub.groupby('TeamID2')['Pred1'].apply(lambda x : (x ** -1.0).mean() ** -1.0 ).fillna(0.5).to_dict()

Any missing value for the prediciton will be imputed with the product of the probabilities calculated above.  We assume these are independent events.

In [153]:
sub['Pred'] = sub[['TeamID1', 'TeamID2','Pred1']]\
.apply(lambda x : team1_probs.get(x[0]) * ( 1 - team2_probs.get(x[1]) ) if np.isnan(x[2]) else x[2], 
       axis = 1)

In [154]:
sub

,TeamID1,TeamID2,Season,Pred,Pred1,ID
0,1101,1113,2019,1.342376e-48,NaN,2019_1101_1113
1,1101,1120,2019,4.160069e-49,NaN,2019_1101_1120
2,1101,1124,2019,6.712710e-49,NaN,2019_1101_1124
3,1101,1125,2019,6.712710e-49,NaN,2019_1101_1125
4,1101,1133,2019,6.712710e-49,NaN,2019_1101_1133
...,...,...,...,...,...,...
11385,1459,1462,2015,5.000000e-01,NaN,2015_1459_1462
11386,1459,1463,2019,5.000000e-01,NaN,2019_1459_1463
11387,1460,1462,2018,5.000000e-01,NaN,2018_1460_1462
11388,1461,1462,2015,5.000000e-01,NaN,2015_1461_1462


In [155]:
sub[['ID', 'Pred']].to_csv('sub.csv', index = False)

In [98]:
sub.Pred = round(sub.Pred*2,0)/2

In [99]:
sub

,ID,Pred,Season,TeamID1,TeamID2,Pred1
0,2015_1107_1112,0.0,2015,1107,1112,NaN
1,2015_1107_1116,0.0,2015,1107,1116,NaN
2,2015_1107_1124,0.0,2015,1107,1124,NaN
3,2015_1107_1125,0.0,2015,1107,1125,NaN
4,2015_1107_1129,0.0,2015,1107,1129,NaN
...,...,...,...,...,...,...
11871,2019_1449_1459,0.5,2019,1449,1459,NaN
11872,2019_1449_1463,0.5,2019,1449,1463,NaN
11873,2019_1458_1459,0.5,2019,1458,1459,NaN
11874,2019_1458_1463,0.5,2019,1458,1463,NaN


In [97]:
sub.iloc[:,:2].to_csv("sub.csv",index=False)

In [99]:
sub

,ID,Pred,Season,TeamID1,TeamID2,Pred1
0,2015_1107_1112,3.171216e-18,2015,1107,1112,NaN
1,2015_1107_1116,3.171216e-18,2015,1107,1116,NaN
2,2015_1107_1124,3.171216e-18,2015,1107,1124,NaN
3,2015_1107_1125,3.171216e-18,2015,1107,1125,NaN
4,2015_1107_1129,3.171216e-18,2015,1107,1129,NaN
...,...,...,...,...,...,...
11385,2019_1449_1459,5.000000e-01,2019,1449,1459,NaN
11386,2019_1449_1463,5.000000e-01,2019,1449,1463,NaN
11387,2019_1458_1459,9.837292e-01,2019,1458,1459,NaN
11388,2019_1458_1463,9.837292e-01,2019,1458,1463,NaN
